In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as si
import time

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers, activations, initializers
from tensorflow.keras.layers import BatchNormalization

np.set_printoptions(precision = 3, suppress = True)
import warnings
warnings.filterwarnings("ignore")

In [2]:
ES_EZ = pd.read_csv('/Users/gaojinglun/Desktop/RSG/ES_EZ.csv')
ES_EZ = ES_EZ.drop(['Unnamed: 0'], axis = 1)
df = ES_EZ[['Put.Call', 'Strike.Price', 'Settlement', 
            'futures.price', 'Time.to.maturity', 'Implied.Volatility']]
df.head()

,Put.Call,Strike.Price,Settlement,futures.price,Time.to.maturity,Implied.Volatility
0,C,2200,1059.0,3259.0,0.213699,0.338267
1,C,2250,1009.1,3259.0,0.213699,0.326829
2,C,2270,989.1,3259.0,0.213699,0.323456
3,C,2300,959.2,3259.0,0.213699,0.317834
4,C,2310,949.2,3259.0,0.213699,0.314282


In [3]:
ES_EZ_call = df[df['Put.Call'] == 'C']
Call_temp = 100 * ES_EZ_call['Strike.Price'].values / ES_EZ_call['futures.price'].values
Call_ITM_idx = Call_temp < 95.2
Call_ATM_idx = np.logical_and(Call_temp < 103, Call_temp >= 95.2)
Call_OTM_idx = Call_temp > 103
print('The number of ITM Call options is', np.sum(Call_ITM_idx))
print('The number of ATM Call options is', np.sum(Call_ATM_idx))
print('The number of OTM Call options is', np.sum(Call_OTM_idx))

The number of ITM Call options is 6678
The number of ATM Call options is 2223
The number of OTM Call options is 4807


In [4]:
# Add the moneyness indicator 
Call_moneyness = np.array(['ITM'] * ES_EZ_call.shape[0])
Call_moneyness[Call_ATM_idx] = 'ATM'
Call_moneyness[Call_OTM_idx] = 'OTM'

ES_EZ_call_X = ES_EZ_call[['futures.price', 'Strike.Price', 'Time.to.maturity']]
ES_EZ_call_y = ES_EZ_call['Implied.Volatility']
ES_EZ_call_X.shape, ES_EZ_call_y.shape

((13708, 3), (13708,))

In [5]:
X_train, X_test, y_train, y_test = train_test_split(ES_EZ_call_X, ES_EZ_call_y, 
                                                    test_size = 0.2, stratify = Call_moneyness, random_state = 123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10966, 3), (2742, 3), (10966,), (2742,))

In [6]:
# Standardize the features
scalerX = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scalerX.transform(X_train), columns = ES_EZ_call_X.columns.values) 
X_test = pd.DataFrame(scalerX.transform(X_test), columns = ES_EZ_call_X.columns.values) 

scalery = MinMaxScaler().fit(y_train.values.reshape(-1, 1))
y_train = scalery.transform(y_train.values.reshape(-1, 1))
y_test = scalery.transform(y_test.values.reshape(-1, 1))
X_train.head()

,futures.price,Strike.Price,Time.to.maturity
0,0.417376,0.485714,0.042980
1,0.441082,0.567347,0.478510
2,0.758969,0.522449,0.083095
3,0.428977,0.424490,0.002865
4,0.633945,0.634694,0.000000


In [7]:
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs = {}):
        self.times = []

    def on_epoch_begin(self, batch, logs = {}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs = {}):
        self.times.append(time.time() - self.epoch_time_start)

In [8]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
score = []
for i in range(10):
    learning_rate = [1e-2, 1e-3]
    score_small = []
    for lr in learning_rate:
        model = models.Sequential()
        # Input layer
        model.add(layers.Dense(units = 3,
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))

        numLayers = 3
        for i in range(numLayers):
            model.add(BatchNormalization())
            model.add(layers.Dense(units = 100, 
                                   kernel_initializer = initializers.RandomNormal(seed = 123),
                                   bias_initializer = initializers.Zeros()))
            model.add(layers.Activation(activations.elu))
            model.add(layers.Dropout(0.25))

        model.add(BatchNormalization())
        model.add(layers.Dense(units = 25, 
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))

        # Output layer
        model.add(layers.Dense(units = 1, activation = "exponential"))

        model.compile(
            optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
        )

        time_callback = TimeHistory()

        history = model.fit(X_train, 
            y_train, 
            epochs = 100, 
            verbose = 0,
            batch_size = 32, 
            callbacks = [earlyStop, time_callback],
            validation_split = 0.2)
        BSNN_train_pred = model.predict(X_train)
        BSNN_test_pred = model.predict(X_test)
        trainr2 = np.round(r2_score(y_train, BSNN_train_pred), 4)
        testr2 = np.round(r2_score(y_test, BSNN_test_pred), 4)
        score_small.append((trainr2, testr2))
    score.append(score_small)
score

[[(-0.6878, -0.6368), (0.8938, 0.8979)],
 [(0.8328, 0.8387), (0.9058, 0.8997)],
 [(0.8023, 0.7949), (0.9003, 0.8985)],
 [(0.7521, 0.7657), (0.9109, 0.9121)],
 [(0.7905, 0.7995), (0.8628, 0.8579)],
 [(0.7526, 0.7599), (0.8979, 0.8891)],
 [(0.856, 0.8564), (0.9118, 0.9156)],
 [(0.812, 0.7954), (0.8549, 0.8606)],
 [(0.9014, 0.9077), (0.9067, 0.9079)],
 [(0.3684, 0.3935), (0.9026, 0.907)]]

In [9]:
learning_rate = [1e-2, 1e-3]
History = []
times = []
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
for lr in learning_rate:
    model = models.Sequential()
    # Input layer
    model.add(layers.Dense(units = 5,
                           kernel_initializer = initializers.RandomNormal(seed = 123),
                           bias_initializer = initializers.Zeros()))
    model.add(layers.Activation(activations.elu))
    
    numLayers = 3
    for i in range(numLayers):
        model.add(BatchNormalization())
        model.add(layers.Dense(units = 100, 
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))
    
    model.add(BatchNormalization())
    model.add(layers.Dense(units = 25, 
                           kernel_initializer = initializers.RandomNormal(seed = 123),
                           bias_initializer = initializers.Zeros()))
    model.add(layers.Activation(activations.elu))
    model.add(layers.Dropout(0.25))

    # Output layer
    model.add(layers.Dense(units = 1, activation = "exponential"))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
    )
    
    time_callback = TimeHistory()

    history = model.fit(X_train, 
        y_train, 
        epochs = 100, 
        verbose = 0,
        batch_size = 32, 
        callbacks = [earlyStop, time_callback],
        validation_split = 0.2)
    BSNN_train_pred = model.predict(X_train)
    BSNN_test_pred = model.predict(X_test)
    print('When the learning rate is', lr)
    trainr2 = np.round(r2_score(y_train, BSNN_train_pred), 4)
    print("The training R squared is", 
          trainr2)
    testr2 = np.round(r2_score(y_test, BSNN_test_pred), 4)
    print("The testing R squared is", 
          testr2)
    print('-------------------------------------------------------------------')
    History.append(history)
    times.append(time_callback.times)
    path = "/Users/gaojinglun/Desktop/RSG/1.5ANN_call_Train_On_ESEZ_Less_Inputs_withLR" + str(lr)
    model.save(path)

When the learning rate is 0.01
The training R squared is 0.8282
The testing R squared is 0.8331
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/1.5ANN_call_Train_On_ESEZ_Less_Inputs_withLR0.01/assets
When the learning rate is 0.001
The training R squared is 0.902
The testing R squared is 0.9042
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/1.5ANN_call_Train_On_ESEZ_Less_Inputs_withLR0.001/assets


In [10]:
ES_EZ_put = df[df['Put.Call'] == 'P']
Put_temp = 100 * ES_EZ_put['Strike.Price'].values / ES_EZ_put['futures.price'].values
Put_ITM_idx = Put_temp > 103
Put_ATM_idx = np.logical_and(Put_temp < 103, Put_temp >= 95.2)
Put_OTM_idx = Put_temp < 95.2
print('The number of ITM Call options is', np.sum(Put_ITM_idx))
print('The number of ATM Call options is', np.sum(Put_ATM_idx))
print('The number of OTM Call options is', np.sum(Put_OTM_idx))

The number of ITM Call options is 3123
The number of ATM Call options is 2234
The number of OTM Call options is 10906


In [11]:
# Add the moneyness indicator 
Put_moneyness = np.array(['ITM'] * ES_EZ_put.shape[0])
Put_moneyness[Put_ATM_idx] = 'ATM'
Put_moneyness[Put_OTM_idx] = 'OTM'

ES_EZ_put_X = ES_EZ_put[['futures.price', 'Strike.Price', 'Time.to.maturity']]
ES_EZ_put_y = ES_EZ_put['Implied.Volatility']
ES_EZ_put_X.shape, ES_EZ_put_y.shape

((16263, 3), (16263,))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(ES_EZ_put_X, ES_EZ_put_y, 
                                                    test_size = 0.2, stratify = Put_moneyness, random_state = 123)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13010, 3), (3253, 3), (13010,), (3253,))

In [13]:
# Standardize the features
scalerX = MinMaxScaler().fit(X_train)
X_train = pd.DataFrame(scalerX.transform(X_train), columns = ES_EZ_call_X.columns.values) 
X_test = pd.DataFrame(scalerX.transform(X_test), columns = ES_EZ_call_X.columns.values) 

scalery = MinMaxScaler().fit(y_train.values.reshape(-1, 1))
y_train = scalery.transform(y_train.values.reshape(-1, 1))
y_test = scalery.transform(y_test.values.reshape(-1, 1))

In [14]:
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
score = []
for i in range(10):
    learning_rate = [1e-2, 1e-3]
    score_small = []
    for lr in learning_rate:
        model = models.Sequential()
        # Input layer
        model.add(layers.Dense(units = 3,
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))

        numLayers = 3
        for i in range(numLayers):
            model.add(BatchNormalization())
            model.add(layers.Dense(units = 100, 
                                   kernel_initializer = initializers.RandomNormal(seed = 123),
                                   bias_initializer = initializers.Zeros()))
            model.add(layers.Activation(activations.elu))
            model.add(layers.Dropout(0.25))

        model.add(BatchNormalization())
        model.add(layers.Dense(units = 25, 
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))

        # Output layer
        model.add(layers.Dense(units = 1, activation = "exponential"))

        model.compile(
            optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
        )

        time_callback = TimeHistory()

        history = model.fit(X_train, 
            y_train, 
            epochs = 100, 
            verbose = 0,
            batch_size = 32, 
            callbacks = [earlyStop, time_callback],
            validation_split = 0.2)
        BSNN_train_pred = model.predict(X_train)
        BSNN_test_pred = model.predict(X_test)
        trainr2 = np.round(r2_score(y_train, BSNN_train_pred), 4)
        testr2 = np.round(r2_score(y_test, BSNN_test_pred), 4)
        score_small.append((trainr2, testr2))
    score.append(score_small)
score

[[(0.8025, 0.8009), (0.9106, 0.9096)],
 [(0.8276, 0.8199), (0.9347, 0.9292)],
 [(0.9142, 0.9047), (0.9333, 0.9268)],
 [(0.8577, 0.8459), (0.9215, 0.9156)],
 [(0.8548, 0.8546), (0.7057, 0.6967)],
 [(0.8573, 0.853), (0.9278, 0.9215)],
 [(0.7456, 0.7382), (0.9253, 0.9184)],
 [(0.7579, 0.7568), (0.9187, 0.9115)],
 [(0.9063, 0.9052), (0.9306, 0.9261)],
 [(0.9006, 0.8924), (0.9003, 0.8947)]]

In [15]:
learning_rate = [1e-2, 1e-3]
History = []
times = []
earlyStop = tf.keras.callbacks.EarlyStopping(monitor = 'loss', patience = 3)
for lr in learning_rate:
    model = models.Sequential()
    # Input layer
    model.add(layers.Dense(units = 3,
                           kernel_initializer = initializers.RandomNormal(seed = 123),
                           bias_initializer = initializers.Zeros()))
    model.add(layers.Activation(activations.elu))
    
    numLayers = 3
    for i in range(numLayers):
        model.add(BatchNormalization())
        model.add(layers.Dense(units = 100, 
                               kernel_initializer = initializers.RandomNormal(seed = 123),
                               bias_initializer = initializers.Zeros()))
        model.add(layers.Activation(activations.elu))
        model.add(layers.Dropout(0.25))
        
    model.add(BatchNormalization())
    model.add(layers.Dense(units = 24, 
                           kernel_initializer = initializers.RandomNormal(seed = 123),
                           bias_initializer = initializers.Zeros()))
    model.add(layers.Activation(activations.elu))
    model.add(layers.Dropout(0.25))

    # Output layer
    model.add(layers.Dense(units = 1, activation = "exponential"))

    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = lr), loss = "mean_squared_error"
    )
    
    time_callback = TimeHistory()

    history = model.fit(X_train, 
        y_train, 
        epochs = 100, 
        verbose = 0,
        batch_size = 32, 
        callbacks = [earlyStop, time_callback],
        validation_split = 0.2)
    BSNN_train_pred = model.predict(X_train)
    BSNN_test_pred = model.predict(X_test)
    print('When the learning rate is', lr)
    print("The training R squared is", 
          np.round(r2_score(y_train, BSNN_train_pred), 4))
    print("The testing R squared is", 
          np.round(r2_score(y_test, BSNN_test_pred), 4))
    print('-------------------------------------------------------------------')
    History.append(history)
    times.append(time_callback.times)
    path = "/Users/gaojinglun/Desktop/RSG/1.5ANN_put_Train_On_ESEZ_Less_Inputs_withLR" + str(lr)
    model.save(path)

When the learning rate is 0.01
The training R squared is 0.8517
The testing R squared is 0.8437
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/1.5ANN_put_Train_On_ESEZ_Less_Inputs_withLR0.01/assets
When the learning rate is 0.001
The training R squared is 0.8969
The testing R squared is 0.8936
-------------------------------------------------------------------
INFO:tensorflow:Assets written to: /Users/gaojinglun/Desktop/RSG/1.5ANN_put_Train_On_ESEZ_Less_Inputs_withLR0.001/assets
